In [1]:
import wave
import torch
# from balacoon_tts import TTS
import time
import threading
from TTS.api import TTS
import requests
import json

/mnt/nlpgridio3/data/awadhawan/s2st/translatotron2/tenv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Balacoon TTS
tts = TTS("models/en_us_hifi92_e2ept.addon")
supported_speakers = tts.get_speakers()
speaker = supported_speakers[-1] if supported_speakers else ""
print(supported_speakers)


NameError: name 'TTS' is not defined

In [7]:
#Coqui TTS
tts = TTS(model_name="tts_models/multilingual/multi-dataset/your_tts", progress_bar=True, gpu=False)

 > Downloading model to /home1/a/anshulw/.local/share/tts/tts_models--multilingual--multi-dataset--your_tts


100%|██████████| 425M/425M [00:24<00:00, 17.3MiB/s] 
/mnt/nlpgridio3/data/awadhawan/s2st/translatotron2/tenv/lib/python3.9/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(


 > Model's license - CC BY-NC-ND 4.0
 > Check https://creativecommons.org/licenses/by-nc-nd/4.0/ for more info.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Model fully restored. 
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:64
 | 

In [2]:
#Eleven Labs
url = "https://api.elevenlabs.io/v1/text-to-speech/TxGEqnHWrfWFTfGW9XjX"
headers = {
    "xi-api-key": "ff79eb4ad43bb12b04a3968a5b92125b"
}

In [3]:
text ="A muffin is an individually portioned baked product, however the term can refer to one of two distinct items: a part-raised flatbread that is baked and then cooked on a griddle, or a quickbread that is chemically leavened and then baked in a mold."
text_small = "Muffin is a baked item, either a flatbread cooked on griddle or a quickbread leavened chemically and baked in a mold."

In [5]:
def streaming_write(sentence):
    for word in sentence.split():
        with open("streaming_input.txt", "a") as file:
            file.write(word+" ")
        time.sleep(0.2)

In [38]:
write_thread = threading.Thread(target=streaming_write, args = (text,) )
write_thread.start()

In [9]:
write_thread = threading.Thread(target=streaming_write, args = (text_small,) )
write_thread.start()

In [10]:
def convert_to_audio(chunk, name):
    # samples = tts.synthesize(chunk, speaker)
    # with wave.open("outputs/example"+ str(name) +".wav", "w") as fp:
    #     fp.setparams((1, 2, 24000, len(samples), "NONE", "NONE"))
    #     fp.writeframes(samples)
    
    # TODO: 16k sample rate
    # tts.tts_to_file(chunk, speaker_wav="inputs/CommonVoiceES.wav", language="en", file_path="outputs/example"+ str(name) +".wav")
    # tts.tts_to_file(chunk, speaker=tts.speakers[0], language="en", file_path="outputs/example"+ str(name) +".wav")
    data = {
        "text": chunk,
        "voice_settings": {
            "stability": 0,
            "similarity_boost": 0
        }
    }

    response = requests.post(url, data=json.dumps(data), headers=headers)

    if response.status_code == 200:
        with open("outputs/example"+ str(name) +".mp3", 'wb') as f:
            f.write(response.content)
    else:
        print("Error")



        
def streamingTTS():
    content=""
    while True:
        with open("streaming_input.txt", "r") as file:
            new_content = file.read()
            print(len(content), len(new_content))
            if len(new_content) > len(content):
                convert_to_audio(new_content[len(content):], len(new_content))
                content = new_content
            else:
                break
        time.sleep(1.2)


In [11]:
tts_thread = threading.Thread(target=streamingTTS)
tts_thread.start()

0 12


12 64
64 118
118 118
